In [1]:
import time
#time.sleep(60*60*3)

In [2]:
import os
# --- Uncomment to use only CPU (e.g. GPU memory is too small)
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# import sys
# sys.path.append("/usr/local/cuda-10.1/bin")
# os.environ["LD_LIBRARY_PATH"] = "/usr/local/cuda-10.1/lib64"

import tensorflow as tf
tf.test.is_gpu_available(cuda_only=True) 

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [3]:
# reate logger - nocer formatting
import logging

logger = logging.getLogger()
logging._warn_preinit_stderr = 0
logger.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s: %(message)s')
ch = logging.StreamHandler()
ch.setFormatter(formatter)
logger.handlers = [ch]
logger.info("Logger started.")

2020-12-31 08:00:03,420: Logger started.


In [4]:
import sqlite3 
import numpy as np

def adapt_array(arr):
    """
    Save Numpy array to SqLite.
    Source:
    http://stackoverflow.com/a/31312102/190597 (SoulNibbler)
    """
    out = io.BytesIO()
    np.save(out, arr)
    out.seek(0)
    return sqlite3.Binary(out.read())

def convert_array(text):
    """ 
    Load Numpy array from Sqlite.
    Source:
    http://stackoverflow.com/a/31312102/190597 (SoulNibbler)
    """
    out = io.BytesIO(text)
    out.seek(0)
    return np.load(out)

sqlite3.register_adapter(np.ndarray, adapt_array)
sqlite3.register_converter("array", convert_array)
data_path = os.path.join(os.getcwd(), "samplesGeneration", "prepared_data.bd")
data_path = "E:/tmp/prepared_data_abs_not_full.bd"
source = sqlite3.connect(data_path, detect_types=sqlite3.PARSE_DECLTYPES, check_same_thread=False)
db = sqlite3.connect(':memory:')
source.backup(db)

In [5]:
#import os
# --- Uncomment to use only CPU (e.g. GPU memory is too small)
#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
#os.environ["CUDA_VISIBLE_DEVICES"] = ""

# import sys
# sys.path.append("/usr/local/cuda-10.1/bin")
# os.environ["LD_LIBRARY_PATH"] = "/usr/local/cuda-10.1/lib64"

#import tensorflow as tf
#tf.test.is_gpu_available(cuda_only=True) 

In [6]:
import io
sqlite3.register_adapter(np.ndarray, adapt_array)
sqlite3.register_converter("array", convert_array)

db_cursor = db.cursor()
sql_query = "SELECT COALESCE(MAX(id)+1, 0) FROM data"
db_cursor.execute(sql_query)
number_of_samples = db_cursor.fetchone()[0]
print("number of samples: ", number_of_samples)

sql_query = "SELECT x, y FROM data WHERE id == 1"
db_cursor.execute(sql_query)

number of samples:  1682261


In [7]:
import tensorflow.keras as keras

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras from SQLite database'
    def __init__(self, path, indexes=None, batch_size=25,
                 shuffle=True, dim=(3, 39, 1), dim_y=(36, 1), column="voxels"):
        'Initialization'
        if indexes is None:
            raise Exception("Indexes need to be provided!")
        sqlite3.register_adapter(np.ndarray, adapt_array)
        sqlite3.register_converter("array", convert_array)
        self.path = path
        self.dim = dim
        self.dim_y = dim_y
        self.db = sqlite3.connect(self.path, detect_types=sqlite3.PARSE_DECLTYPES, check_same_thread=False)
        self.db_cursor = self.db.cursor()
        self.N = len(indexes) - 1
        self.sample_index = indexes
        self.column = column
        self.batch_size = int(batch_size)
        self.shuffle = shuffle
        self.on_epoch_end()
        self.db.close()


    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(self.N / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data - generates indexes of the batch'
        # Generate indexes of the batch
        samples_batch = np.arange((index) * self.batch_size, (index+1) * self.batch_size)

        # Generate data
        while True:
            try:
                x, y = self.__data_generation(samples_batch)
                break
            except ValueError:
                continue

        return  x, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        if self.shuffle:
            np.random.shuffle(self.sample_index)

    def __data_generation(self, samples_batch):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        x = np.empty((self.batch_size, *self.dim))
        y = np.empty(shape=(self.batch_size, *self.dim_y))
        inds = self.sample_index[samples_batch]
        sqlite3.register_adapter(np.ndarray, self.adapt_array)
        sqlite3.register_converter("array", self.convert_array)
        db = sqlite3.connect(self.path, detect_types=sqlite3.PARSE_DECLTYPES)
        db_cursor = db.cursor()
        sql_query = "SELECT x, y FROM data WHERE id in ({index})".\
            format(column=self.column, index=','.join(str(ind) for ind in inds))
        db_cursor.execute(sql_query)
        try:
            for i, line in zip(range(self.batch_size), db_cursor.fetchall()):
                if line is None:
                    x[i, :] = x[i - 1, :]
                    y[i] = y[i - 1]
                    continue # Bad, temporary solution
                x[i, :] = np.array(np.array_split(line[0].reshape(self.dim), 3)).reshape(self.dim)
                y[i] = line[1].reshape(self.dim_y)
        except TypeError as e:
            print(sql_query)
            raise e
        # x.reshape(self.batch_size, *self.dim, 1)
        y = y.reshape(self.batch_size, *self.dim_y)
        return x, y
    
    def adapt_array(self, arr):
        """
        Save Numpy array to SqLite.
        Source:
        http://stackoverflow.com/a/31312102/190597 (SoulNibbler)
        """
        out = io.BytesIO()
        np.save(out, arr)
        out.seek(0)
        return sqlite3.Binary(out.read())

    def convert_array(self, text):
        """ 
        Load Numpy array from Sqlite.
        Source:
        http://stackoverflow.com/a/31312102/190597 (SoulNibbler)
        """
        out = io.BytesIO(text)
        out.seek(0)
        return np.load(out)
    
test_train_ratio = 0.1

test_size = int(test_train_ratio * number_of_samples)

indexes = np.arange(0, number_of_samples - 1)
np.random.shuffle(indexes)
train_indexes, val_indexes = indexes[:-test_size], indexes[test_size:]

training_generator = DataGenerator(data_path, indexes=train_indexes, batch_size=8, dim=(3, 39), dim_y=(1, 36))
validation_generator = DataGenerator(data_path, indexes=val_indexes, batch_size=8, dim=(3, 39), dim_y=(1, 36))

In [8]:
import time
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

best_model_file = "model.h5"
best_model = ModelCheckpoint(best_model_file, monitor='val_loss', mode='min',verbose=1, save_best_only=True)

models_dir = os.path.join(os.getcwd(), "samplesGeneration", "logs")

In [9]:
class CustomSaveCallback(keras.callbacks.Callback):

    def on_test_begin(self, logs=None):
        self.model.save("model\\improved_model_wip_abs2", save_format='tf')
my_cb = CustomSaveCallback()

In [ ]:
# Try learning for all of verts:

verts = range(14)
pivot_vert = 3

from tensorflow.keras import Input, Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop
from collections import deque
from tensorflow.keras.callbacks import EarlyStopping
import time
import tensorflow.keras.backend as K

models_dir = os.path.join(os.getcwd(), "samplesGeneration", "logs", "model_{}".format(time.time()))

def special_loss(y_true, y_pred):
    custom_loss=tf.add(tf.square(y_true-y_pred)/tf.add(tf.square(y_pred), 0.1), tf.keras.losses.MSE(y_true, y_pred))
    return custom_loss    

def loss(y_true,y_pred):
    return K.sum(K.square(y_pred - y_true) * K.square(100 * y_true) + K.square(y_pred - y_true))

physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)
    
def create_and_train_network(generator, val_generator, epohs=10):
    log_dir = os.path.join(os.getcwd(), "samplesGeneration", "logs", "model_{}".format(time.time()))
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
    
    def create_model():
        input_1 = Input(shape=generator.dim, name='input_1')
        x = LSTM(np.prod(generator.dim_y), return_sequences=False, dropout=0.2)(input_1)
        output_1 = Reshape((np.prod(generator.dim_y), 1))(x)
        model = Model(inputs=[input_1], outputs=[output_1])
        model.compile(loss="mse", optimizer='rmsprop')
        return model

    logger.info("Create model...")
    model = create_model()
    model.summary()
    logger.info("Fit model..")
    network = model.fit_generator(training_generator, verbose=1, validation_data=validation_generator, 
                                  epochs=epohs, callbacks=[my_cb, early_stopping, best_model])
    logger.info("Summary")
    model.summary()
    model.save(os.path.join(models_dir, "model"))
    return network


logger.info("start")
network = create_and_train_network(training_generator, validation_generator)

2020-12-31 09:39:14,276: start
2020-12-31 09:39:14,281: Create model...
2020-12-31 09:39:14,520: Fit model..


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 3, 39)]           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 36)                10944     
_________________________________________________________________
reshape_1 (Reshape)          (None, 36, 1)             0         
Total params: 10,944
Trainable params: 10,944
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10


D:\ProgramData\Anaconda3\envs\tf38\lib\site-packages\tensorflow\python\keras\engine\training.py:1879: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


189246/189254 [============================>.] - ETA: 0s - loss: 9.5551e-04

2020-12-31 09:53:50,461: Found untraced functions such as lstm_cell_2_layer_call_and_return_conditional_losses, lstm_cell_2_layer_call_fn, lstm_cell_2_layer_call_fn, lstm_cell_2_layer_call_and_return_conditional_losses, lstm_cell_2_layer_call_and_return_conditional_losses while saving (showing 5 of 5). These functions will not be directly callable after loading.


...

189254/189254 [==============================] - 5082s 27ms/step - loss: 8.3949e-04 - val_loss: 8.3910e-04

Epoch 00005: val_loss improved from 0.00084 to 0.00084, saving model to model.h5
Epoch 6/10
189245/189254 [============================>.] - ETA: 0s - loss: 8.4286e-04

2020-12-31 12:59:57,800: Found untraced functions such as lstm_cell_2_layer_call_and_return_conditional_losses, lstm_cell_2_layer_call_fn, lstm_cell_2_layer_call_fn, lstm_cell_2_layer_call_and_return_conditional_losses, lstm_cell_2_layer_call_and_return_conditional_losses while saving (showing 5 of 5). These functions will not be directly callable after loading.


INFO:tensorflow:Assets written to: model\improved_model_wip2\assets


2020-12-31 12:59:59,832: Assets written to: model\improved_model_wip2\assets


189254/189254 [==============================] - 1633s 9ms/step - loss: 8.4286e-04 - val_loss: 8.4007e-04

Epoch 00006: val_loss did not improve from 0.00084
Epoch 7/10
  1941/189254 [..............................] - ETA: 13:02 - loss: 7.9213e-04

In [11]:
model = tf.keras.models.load_model(best_model_file, custom_objects={"loss": loss})

model.save("model\\improved_model_wip_abs2", save_format='tf')

2020-12-31 09:38:44,994: Found untraced functions such as lstm_cell_1_layer_call_and_return_conditional_losses, lstm_cell_1_layer_call_fn, lstm_cell_1_layer_call_fn, lstm_cell_1_layer_call_and_return_conditional_losses, lstm_cell_1_layer_call_and_return_conditional_losses while saving (showing 5 of 5). These functions will not be directly callable after loading.


INFO:tensorflow:Assets written to: model\improved_model_wip2\assets


2020-12-31 09:38:46,309: Assets written to: model\improved_model_wip2\assets


In [ ]:
saver = tf.train.Saver()

In [ ]:
import io

sqlite3.register_adapter(np.ndarray, adapt_array)
sqlite3.register_converter("array", convert_array)
db = sqlite3.connect(data_path, detect_types=sqlite3.PARSE_DECLTYPES)
db_cursor = db.cursor()
sql_query = "SELECT x, y FROM data WHERE id == 1"
db_cursor.execute(sql_query)
data = db_cursor.fetchone()[0]
print(data.shape)
predicted= list()
frames = np.array(np.array_split(data, 3))
for f in frames:
    predicted.append(f)

for n in range(240):
    _in  = np.concatenate((predicted[-3], predicted[-2], predicted[-1])).reshape(116, 1)
    pred = model.predict(_in)
    p = (n ==5) *10
    np.append(pred, p)
    np.append(pred, data[-2])
    np.append(pred, data[-1])
    print(pred.shape)
    predicted.append(pred)

In [ ]:
import matplotlib as plt

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d', xlim=(-2,2),ylim=(-2,2), zlim=(0, 10))

xs = list()
ys = list()
zs = list()
sizes = list()
sample_to_plot = data[600]
for num, f in enumerate(sample_to_plot):
    for i in range(4):
        v = [f[9*i], f[9*i + 1], f[9*i + 2]]
        # print(v)
        xs.append(v[0])
        ys.append(v[2])
        zs.append(v[1])
        sizes.append([(len(sample_to_plot) - float(num))/len(sample_to_plot), 0, 0])
ax.scatter(xs, ys, zs, c=sizes)

plt.show()